In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
import os
%matplotlib inline

warnings.filterwarnings("ignore") #, category=DeprecationWarning)

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

In [3]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

# app.config['SQLALCHEMY_DATABASE_URI'] = DB_URL
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # silence the deprecation warning

# db = SQLAlchemy(app)

In [4]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [5]:
# TODO Fix SQL input/output; temp fix:
df = pd.read_pickle('data/kickstarter_data.pkl')

In [6]:
print(df.shape)
df.head()

(19291, 20)


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved,main_category_code,country_code,currency_code,state_code
ID,,,,,,,,,,,,,,,,,,,,
1031685482,You Sparkle Inside: A Kids' Book + Commissions...,Children's Books,Publishing,USD,2018-01-02,2000.0,2017-12-06,6083.00,successful,133,US,1054.0,6083.00,2000.0,27 days,304.150000,12,21,13,1
1176736360,THE NEW BILLY WYLDER RECORD,Indie Rock,Music,USD,2018-01-02,10000.0,2017-11-30,11169.56,successful,177,US,650.0,11169.56,10000.0,33 days,111.695600,10,21,13,1
1894703097,Driftwood Holly ***The Venice Project***,Indie Rock,Music,EUR,2018-01-02,25000.0,2017-11-28,25417.20,successful,168,DE,0.0,30615.02,30112.5,35 days,101.668809,10,5,4,1
311863059,On a Limb's debut album,Jazz,Music,USD,2018-01-02,1000.0,2017-12-09,1743.00,successful,39,US,0.0,1743.00,1000.0,24 days,174.300000,10,21,13,1
2076121120,Briggs Barbecue Sauces,Small Batch,Food,USD,2018-01-02,200000.0,2017-11-03,1.00,failed,1,US,0.0,1.00,200000.0,60 days,0.000500,7,21,13,0


In [7]:
X = df[['main_category_code','country_code','currency_code','usd_goal_real']]
y = df['state_code']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=42)

k_range = list(range(1, 101))
print(X.shape)
print(y.shape)

(19291, 4)
(19291,)


## Decision Trees

In [8]:
# HOW do I tie this analysis into my project?  I don't understand how to incorporate y1
y1 = y_test # I have a hard time interpreting random data into project scenario

import matplotlib.cm as cm


def visualize(X, y):
    c = cm.rainbow(np.linspace(0, 1, 2))
    plt.scatter([i[0] for i in X], [i[1] for i in X], color=[c[i] for i in y], alpha=.5)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.grid(True)
    

visualize(X, y1)

NameError: name 'y_test' is not defined

In [ ]:
def quick_test(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
    model.fit(xtrain, ytrain)
    return model.score(xtest, ytest)

def quick_test_afew_times(model, X, y, n=10):
    return np.mean([quick_test(model, X, y) for j in range(n)])

linearsvc = LinearSVC()
# Do the test 10 times with a LinearSVC and get the average score
quick_test_afew_times(linearsvc, X, y)

In [ ]:
decisiontree = DecisionTreeClassifier(max_depth=2)
quick_test_afew_times(decisiontree, X, y1)

In [ ]:
randomforest = RandomForestClassifier()
quick_test_afew_times(randomforest, X, y1)

In [ ]:
y2 = [1 if (0.5-i[0])*(0.5-i[1])>0 else 0 for i in X]
visualize(X, y2)

In [ ]:
decisiontree = DecisionTreeClassifier()
quick_test_afew_times(decisiontree, X, y2)

In [ ]:
linearsvc = LinearSVC()
quick_test_afew_times(linearsvc, X, y2)

In [ ]:
svc = SVC()
quick_test_afew_times(svc, X, y2)

In [ ]:
s2 = SVC()
X2 = (0.5-X) * 2
quick_test_afew_times(s2, X2, y2)